pima-indians-diabetes.csv 데이터를 활용하여 MLP, CNN, RNN을 만들었습니다. 

책에서 데이터의 코드 accuracy 결과를 보면(11과) 약 72%로 나오고 있어 이보다 더 높은 수치를 내는 것을 목표로 하였습니다.

# MLP

*Accuracy: 0.7446*

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import tensorflow as tf

#데이터 적용
my_data = "C:\\Users\\user\\Desktop\\dataset\\pima-indians-diabetes.csv"
dataset = np.loadtxt(my_data, delimiter=",")
X = dataset[:, 0:8]
Y = dataset[:, 8]

#학습셋과 테스트셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

MLP에서는 은닉층을 추가하여 총 3개의 은닉층이 존재하고, 인풋 노드 수를 50으로 늘렸습니다

Dropout layer은 제거하는 것이 더 accuracy가 높게 나왔습니다.

In [80]:
#모델의 설정
model = Sequential()
model.add(Dense(40, input_dim=8, activation='relu')) #1차원 데이터이므로 dense 전에 Flatten을 할 필요가 없음
model.add(Dense(20, activation='relu')) 
model.add(Dense(10, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) #0과 1로 이루어진 binary 데이터이므로 sigmoid를 사용함

#모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #이항 분류이므로 binary crossentropy를 사용함

에포크를 늘리고 batch_size를 추가하였습니다.

In [81]:
model.summary()

#모델 실행
model.fit(X_train, Y_train, epochs=100, batch_size=10)

#결과출력
print("\n Accuracy: %.4f" % model.evaluate(X_test, Y_test)[1])

Model: "sequential_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_124 (Dense)            (None, 40)                360       
_________________________________________________________________
dense_125 (Dense)            (None, 20)                820       
_________________________________________________________________
dense_126 (Dense)            (None, 10)                210       
_________________________________________________________________
dense_127 (Dense)            (None, 1)                 11        
Total params: 1,401
Trainable params: 1,401
Non-trainable params: 0
_________________________________________________________________
Train on 537 samples
Epoch 1/100
537/537 [==============================] - 1s 1ms/sample - loss: 2.1801 - accuracy: 0.5196
Epoch 2/100
537/537 [==============================] - 0s 191us/sample - loss: 0.7293 - accuracy: 0.5996
Epoch 3/100
537/537 

# CNN

*Accuracy: 0.7359*

수업시간에는 mnist 데이터로 Conv2D를 활용하여 cnn을 만드는 방법을 배웠지만, 제가 사용한 indian 데이터는 1차원적 숫자 데이터이므로 접근을 달리 해야 된다고 생각했습니다. 

https://cs.stackexchange.com/questions/82027/can-cnn-be-used-for-numeric-data
해당 링크의 글을 참고하였을 때 이미지가 아닌 숫자 데이터를 다루기 때문에 Conv2D보다는 Conv1D가 적합하다고 판단하여 Conv1D을 적용하였습니다.

여러 시도를 해보았지만 accuracy가 최대 0.7359로 그리 높지 않은 것을 보아 Conv1D 모델을 indian 데이터에 적합한 모델이라고 판단할 수는 없을 것 같습니다.

In [ ]:
#Accuracy 0.7359

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

#seed값 설정
seed=0
np.random.seed(seed)
tf.random.set_seed(3)

#데이터 적용
my_data = "C:\\Users\\user\\Desktop\\dataset\\pima-indians-diabetes.csv"
dataset = np.loadtxt(my_data, delimiter=",")
X = dataset[:, 0:8]
Y = dataset[:, 8]

#학습셋과 테스트셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

Conv1D 모델은 처음 만들어보는 것이라 우여곡절이 있었습니다. 

특히 csv의 숫자 데이터셋의 차원을 reshaping하는 것이 mnist 이미지 데이터와 달라 고민했습니다.

Conv2D는 X_train.reshape(X_train.shape[0], 28, 28, 1)과 같은 차원으로 표현되는 반면, 

Conv1D는 X_train.reshape((X_train.shape[0], X_train.shape[1], 1)과 같은 차원으로 표현하여 차이가 있었습니다.


In [ ]:
#데이터를 reshaping
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1)) 
#데이터의 class가 0 or 1이므로 따로 원핫 인코딩을 적용할 필요가 없음

Conv1D는 은닉층을 생성할 때 input node를 따로 추가하지 않고, kernel_size를 1차원으로, input_shape를 2차원으로 추가하였습니다.

기존의 모델에 비해 은닉층을 하나 더 추가하였고, 필터의 크기를 크게 할수록 accuracy가 높게 나왔습니다.

In [ ]:
#모델 설정
model = Sequential()
model.add(Conv1D(400, kernel_size=3, input_shape=(X_train.shape[1], 1), activation='relu'))
model.add(Conv1D(800, kernel_size=3, activation='relu'))
model.add(Conv1D(800, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid')) 
#이항 분류이므로 활성화함수로 sigmoid를 사용함
#마지막에는 값이 1개가 도출되어야 하기에 노드를 1로 설정함
          
#모델 컴파일
model.compile(loss='binary_crossentropy', #이항분류이므로 binary crossentropy 사용
             optimizer='adam',
             metrics=['accuracy'])

epoch를 더 크게 할수록 accuracy가 높게 나왔습니다. 
그렇지만 epoch가 100을 넘은 이후로는 accuracy가 비슷하게 나와 100으로 설정했습니다. 

batch_size도 추가했습니다.

In [25]:
model.summary()

#모델 실행
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=10)

#모델 평가
model.evaluate(X_test, Y_test)
print("\n Accuracy: %.4f" % model.evaluate(X_test, Y_test)[1])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_38 (Conv1D)           (None, 6, 400)            1600      
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 4, 800)            960800    
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 4, 800)            640800    
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 2, 800)            0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 2, 800)            0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 1600)              0         
_________________________________________________________________
dense_45 (Dense)             (None, 200)             

# RNN

*Accurcy: 0.7489*

MLP, CNN, RNN 중에서 제일 accuracy가 높게 나왔습니다.

RNN은 주로 sequential data에 많이 쓰인다고 알고 있는데, 숫자형 데이터를 다룰 때 이미지형 아키텍쳐인 CNN보다는 텍스트용 sequential 데이터용인 RNN 아키텍쳐가 더 적절하다는 것을 알 수 있었습니다.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

import numpy as np
import tensorflow as tf

#seed값 설정
seed=0
np.random.seed(seed)
tf.random.set_seed(3)

#데이터 적용
my_data = "C:\\Users\\user\\Desktop\\dataset\\pima-indians-diabetes.csv"
dataset = np.loadtxt(my_data, delimiter=",")
X = dataset[:, 0:8]
Y = dataset[:, 8]

#학습셋과 테스트셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=seed)

CNN과 마찬가지로 데이터 reshaping에 있어서 mnist 데이터와 차원을 다르게 하였습니다.

In [ ]:
#데이터를 reshaping 
#mnist 데이터와 다르다!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_train.shape[1], 1)) 

LSTM에서는 노드의 개수를 전반적으로 늘리고, dropout의 퍼센트를 줄였다가 점차 커지도록 하였습니다.

데이터가 binary하므로 마지막 dense 레이어에서 노드를 1개로 설정하고, sigmoid를 사용하였습니다.
또한 sparse categorical crossentropy 대신 binary_crossentropy를 사용하였습니다.

In [ ]:
#모델 설정
model = Sequential()
model.add(LSTM(400, input_shape=(X_train.shape[1], 1), activation='relu', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid')) #출력층에서는 노드가 하나이므로 1을 넣고, 이항 분류이므로 sigmoid와 binary crossentropy 사용
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

epoch를 100으로 늘리고 batch_size를 추가했습니다. epoch를 200으로 설정했을 때 accuracy가 100과 같아 100으로 설정하였습니다.

In [49]:
model.summary()

#모델 실행
model.fit(X_train, Y_train, validation_data = (X_test, Y_test), epochs=100, batch_size=10)

#모델 평가
model.evaluate(X_test, Y_test)
print("\n Accuracy: %.4f" % model.evaluate(X_test, Y_test)[1])

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 8, 400)            643200    
_________________________________________________________________
dropout_48 (Dropout)         (None, 8, 400)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 200)               480800    
_________________________________________________________________
dropout_49 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 40)                8040      
_________________________________________________________________
dropout_50 (Dropout)         (None, 40)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 1)               